# Regresión lineal

## Comenzar definiendo las funciones para la Regresión Logística

**Recordando la hipótesis en regresión lineal:**<br>
$h_\theta = \theta_0 + \theta_1x$<br>
$h_\theta = \theta_0x_0 + \theta_1x_1$<br>
<br>Si hubiera más de una variable:<br>
$h_\theta = \theta_0x_0 + \theta_1x_1 + \theta_2x_2 + ... + \theta_nx_n$<br>
<br>Pero en general:<br>
$h_\theta = \theta^Tx$

**Hipótesis para regresión <font color='orange'>logística:</font>**<br>
$z = \theta^Tx$<br><br>
$\begin{align}
h_\theta = \frac{1}{1+\mathrm{e}^{-z}}
\end{align}$<br><br>
$\begin{align}
h_\theta = \frac{1}{1+\mathrm{e}^{-\theta^Tx}}
\end{align}$

<br>**Función de costo:**<br>
$\begin{align}
J(\theta) = -\frac{1}{m}\sum_{i=1}^m\left[\mathbf{y}^i\log{h_\theta(\mathbf{x}^i)} + (1-\mathbf{y}^i) \log(1-{h_\theta(\mathbf{x}^i)})\right]
\end{align}$<br><br>

<br>**Descenso de gradiente:**<br>
$\begin{align}
    \text{Repetir hasta convergencia: }\\[1em]
    \theta_j := \theta_j - \alpha \frac{\partial}{\partial\theta_j}J(\theta)
\end{align}$

donde su derivada es (la misma que en R. lineal)<br>

y por lo tanto, el descenso de gradiente:<br>
$\begin{align}
    \text{Repetir hasta convergencia: }\\[1em]
    \theta_j := \theta_j - \alpha \frac{1}{m}\sum_{i=1}^m \left(h_\theta(\mathbf{x}^i) - \mathbf{y}^i\right)x^i_j
\end{align}$

## Importar bibliotecas de funciones
Importar bibliotecas, leer los datos y normalizar, según sea el caso.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets

# Leer los datos
data = datasets.load_iris()
yy = data.target
X = data.data[yy!=0,:]
y = data.target[yy!=0] #Para tomar las clases 1 y 2
y[y==2]=0 # A la clase 2 se le renombra como clase 0
patterns, features = X.shape


# Mostrar las 3 clases juntas
plt.scatter(X[0:50,0],X[0:50,3], s=50)
plt.scatter(X[50:100,0],X[50:100,3], s=75)
plt.xlabel('Long sépalo', fontsize=12)
plt.ylabel('Ancho pétalo', fontsize=12)
plt.title('Clasificación con scikit-learn')
plt.show()



In [ ]:
# ¿Normalizar?
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler().fit(X)
X = norm.transform(X)
# print(X)

# Agregar un 1 en la columna del inicio
Xo = np.c_[np.ones(X.shape[0]),X]
# print(Xo)

## Para programar nuestra Regresión logística
### Definir funciones y parámetros iniciales

In [ ]:
# Importar bibliotecas de funciones

import numpy as np
from matplotlib import pyplot as plt

# Definir funciones de R. logística
def h(X,theta):
  z = np.dot(X, theta)
  return 1/(1+np.exp(-z))


def J(X,theta):
  er = -y*np.log(h(X,theta)) - (1-y)*np.log(1-h(X,theta))
  #return np.round(np.sum(er)/(2*X.shape[0]),0)
  return np.sum(er)/(2*X.shape[0])


def dg(X,y,theta,alpha,epochs=10,verbose=False): #Descenso de gradiente
  m = X.shape[0]
  aux = theta.copy()
  er=[]
  for epoch in range(epochs):
    aux[0] = theta[0] - ((alpha/m) * np.sum((h(X,theta) - y) * X[:,0]))
    aux[1] = theta[1] - ((alpha/m) * np.sum((h(X,theta) - y) * X[:,1]))
    theta = aux
    er = np.append(er, J(X,theta))
    if verbose:
      print(f'Iteración {epoch+1}: {np.round(h(X,theta),2)}\n')
  return theta, er

def dg2(X,y,theta,alpha,epochs=10): #Descenso de gradiente
  m = X.shape[0]
  aux = theta.copy()
  er=[]
  for epoch in range(epochs):
    for j in range(len(theta)):
      aux[j] = theta[j] - ((alpha/m) * np.sum((h(X,theta) - y) * X[:,j]))
    theta=aux
    er = np.append(er, J(X,theta))
  return theta, er

In [ ]:
# Parámetros
theta, epochs = np.array([-2.,1.,-2.,1.,1.5]), 250
# theta, epochs = np.random.rand(Xo.shape[1]), 250
alpha = 0.3

# Condiciones iniciales:
yw = h(Xo,theta)
e = J(Xo,theta)
print('y verdadera:\n',y)
print('\nyw inicial:\n', np.round(yw))
print('\nerror inicial:',e)
print('theta inicial:',theta)
print('alpha:',alpha)

### Aplicar la regresión logística

In [ ]:
# La regresión logística

# Parámetros
# theta, epochs = np.array([-2.,1.,-2.,1.,1.5]), 250
theta, epochs = np.random.rand(Xo.shape[1]), 500
alpha = 0.3

th2, er2 = dg2(Xo,y,theta,alpha,epochs=epochs)
yw2 = np.round(h(Xo,th2))
print(f'Nuevo theta dg2: {th2} en {epochs} epochs, con error {J(Xo,th2)}')
print('Accuracy con dg2:',100*(1-np.sum(abs(y-yw2))/X.shape[0]))

# Mapas de color para visualización

from matplotlib.colors import ListedColormap

cmap_light=ListedColormap(['#AAFFAA','#AAAAFF'])
cmap_bold=ListedColormap(['#00FF00','#0000FF'])

# Graficar clasificación
hip = np.round(h(Xo,th2),0)
plt.scatter(X[:,1],X[:,3],c=y, cmap=cmap_bold, s=150, edgecolor='k')
plt.scatter(X[:,1],X[:,3],c=yw2, cmap=cmap_light, s=30)#, edgecolor='m')
plt.xlabel('Long sépalo', fontsize=12)
plt.ylabel('Ancho pétalo', fontsize=12)
plt.title('Clasificación')
plt.show()

# Graficar error:
ep1 = []
ep2 = []
for i in range(epochs):
  ep1.append(i)
for i in range(epochs):
  ep2.append(i)

plt.figure()
plt.plot(ep2,er2)
plt.xlabel('Epoch')
plt.ylabel('Error J(theta)')
plt.title('Descenso de gradiente v2')

plt.show()

## Utilizando scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression().fit(X, y)
ylr = reg.predict(X)
thetaskl = np.append(reg.intercept_, reg.coef_)
sklerror = J(Xo,thetaskl)
print(f'Nuevo theta en sklearn: {thetaskl}; error: {sklerror}.')
print('Accuracy con scikit-learn:',100*(1-np.sum(abs(y-ylr))/X.shape[0]))



### Gráficas

In [ ]:
# Mapas de color para visualización
from matplotlib.colors import ListedColormap

cmap_light=ListedColormap(['#AAFFAA','#AAAAFF'])
cmap_bold=ListedColormap(['#00FF00','#0000FF'])

# Graficar clasificación
hip = np.round(h(Xo,thetaskl),0)
plt.scatter(X[:,1],X[:,3],c=y, cmap=cmap_bold, s=150, edgecolor='k')
plt.scatter(X[:,1],X[:,3],c=ylr, cmap=cmap_light, s=30)#, edgecolor='m')
plt.xlabel('Long sépalo', fontsize=12)
plt.ylabel('Ancho pétalo', fontsize=12)
plt.title('Clasificación con scikit-learn')
plt.show()

In [ ]:
# Graficar fronteras de decisión
delta = 0.02
feat = [1,3]
XX = X[:,feat]
reg = LogisticRegression().fit(XX, y)
x_min, x_max = XX[:, 0].min() - 0.5, XX[:, 0].max() + 0.5
y_min, y_max = XX[:, 1].min() - 0.5, XX[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, delta), np.arange(y_min, y_max, delta))
Z = reg.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, cmap=cmap_light)

plt.scatter(X[:,1],X[:,3],c=y, cmap=cmap_bold, s=150, edgecolor='k')
plt.scatter(X[:,1],X[:,3],c=ylr, cmap=cmap_light, s=30)#, edgecolor='m')
plt.xlabel('Long sépalo', fontsize=12)
plt.ylabel('Ancho pétalo', fontsize=12)
plt.title('Frontera de decisión')
plt.show()